<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#loading" data-toc-modified-id="loading-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>loading</a></span></li><li><span><a href="#modeling" data-toc-modified-id="modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>modeling</a></span></li><li><span><a href="#pred" data-toc-modified-id="pred-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>pred</a></span></li></ul></div>

In [1]:
import sys
import os
my_module = os.path.abspath("../src/")
sys.path.append(my_module)

In [2]:
from load_data import LoadData
# from nklearn.lgbm_util.cv import CvEarlyStopping
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt


# loading

In [3]:
docs = "../docs/"
info = docs + "info/"
finfo = info + "ddl.csv"

data = "../data/"
data_in = data + "in/"
data_out = data + "out/"
fin = data_in + "train_data.csv"
ftest = data_in + "test_data.csv"
fout = data_out + "submit_rf.csv"

loader = LoadData(fin, finfo)
X, y = loader.load()

In [4]:
X = X.astype(float)
y = y.astype(float)

# modeling

In [5]:
params = {'n_estimators'  : [3, 10, 100], 'n_jobs': [-1]}

In [6]:
cv = GridSearchCV(RandomForestClassifier(), param_grid=params, cv=5, scoring='accuracy', n_jobs=-1)

In [7]:
cv.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [3, 10, 100], 'n_jobs': [-1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [29]:
cv.best_score_

0.8141851851851852

In [12]:
from sklearn.metrics import roc_curve

array([0.78, 0.61, 0.03, ..., 0.88, 0.66, 0.69])

In [16]:
fpr, tpr, thresholds = roc_curve(y_true=y, y_score=cv.predict_proba(X)[:,1])

In [17]:
l = (tpr - 1)**2 + fpr**2

In [18]:
rc = pd.DataFrame({
    "l":l,
    "thre":thresholds
})

In [19]:
thre = float(rc[rc.l == rc.l.min()].thre)
thre

0.4139166666666667

# pred

In [21]:
loader = LoadData(ftest, finfo)
test_X= loader.load()

In [22]:
test_X = test_X.astype(float)

In [23]:
pred = cv.predict(test_X)

In [24]:
pred = pred > thre

In [25]:
converter = lambda p: 1 if p else 0


In [26]:
ix = test_X.index

In [27]:
test_X["Y"] = np.vectorize(converter)(pred)

In [28]:
test_X["Y"].to_csv(fout, header=True)